# Parameters guide

Here we will describe the AIBECS interface.
This guide will take you through some examples of setting up model parameters.

> **Note:**
> The parameters features in AIBECS essentially come from other packages.
> These include [Parameters.jl](https://github.com/mauro3/Parameters.jl), [FieldMetadata.jl](https://github.com/rafaqz/FieldMetadata.jl), [Flatten.jl](https://github.com/rafaqz/Flatten.jl), [Unitful.jl](https://github.com/PainterQubits/Unitful.jl).

As usual, make sure you are using AIBECS

In [1]:
using AIBECS

## Abstract parameters type

The AIBECS provides a set of features to create and use parameters.
This features are implemented as functions that must be provided with a parameters type.
But each set of parameters is different, and AIBECS cannot know beforehand what parameters you want to use.
The AIBECS thus provides an abstract parameters type, called `AbstractParameters`, upon which all the AIBECS functionality is built.
This is why when you create a set of parameters in AIBECS, you must declare it as a subtype of `AbstractParameters`.
Here is an example.

In [2]:
struct SimpleParams{T} <: AbstractParameters{T}
    α::T
    β::T
end

Once the type, which here simply defines the symbols (`α` and `β`), is constructed, we can *instantiate* a parameter variable.

In [3]:
p = SimpleParams(1.0, 2.0)


│ Row │ Symbol │ Value   │
│     │ Symbol │ Float64 │
├─────┼────────┼─────────┤
│ 1   │ α      │ 1.0     │
│ 2   │ β      │ 2.0     │

As you can see, the AIBECS will display `p` as a table.
This is because the `show` method converts `p` to a table (a `DataFrame` to be specific) under the hood:

In [4]:
AIBECS.table(p)

,Symbol,Value
,Symbol,Float64
1,α,1.0
2,β,2.0


If you do not remember the order in which you created the parameters (`α` is first, `β` is second), AIBECS has got your back: keyword arguments are supported.

In [5]:
p = SimpleParams(β = 2.0, α = 1.0)


│ Row │ Symbol │ Value   │
│     │ Symbol │ Float64 │
├─────┼────────┼─────────┤
│ 1   │ α      │ 1.0     │
│ 2   │ β      │ 2.0     │

## Unpacking

Probably the most useful feature in AIBECS is the ability to elegantly unpack parameters, thanks to [Parameters.jl](https://github.com/mauro3/Parameters.jl).

In [6]:
@unpack α, β = p
α, β

(1.0, 2.0)

unpacks the parameters on the left (`α` and `β`) from the parameter type `p`.

## Units

One of the main features of parameters in AIBECS is that you can use units and let AIBECS do the conversions for you.
Before you use units, though, you **must** import the `@units` and `units` functions from AIBECS.
Here is an example.

In [7]:
import AIBECS: @units, units
@units struct UnitfulParams{T} <: AbstractParameters{T}
    α::T | u"m/s"
    β::T | u"d"
end

units (generic function with 11 methods)

Cretaing an instance is just as easy

In [8]:
p = UnitfulParams(1.0, 2.0)


│ Row │ Symbol │ Value   │ Unit     │
│     │ Symbol │ Float64 │ Unitful… │
├─────┼────────┼─────────┼──────────┤
│ 1   │ α      │ 1.0     │ m s⁻¹    │
│ 2   │ β      │ 2.0     │ d        │

And in this case the parameters are shown with units.
You can rely on AIBECS to convert units on the fly.

In [9]:
p = UnitfulParams(3.0u"km/hr", 24.0u"hr")


│ Row │ Symbol │ Value    │ Unit     │
│     │ Symbol │ Float64  │ Unitful… │
├─────┼────────┼──────────┼──────────┤
│ 1   │ α      │ 0.833333 │ m s⁻¹    │
│ 2   │ β      │ 1.0      │ d        │

And use keyword arguments

In [10]:
p = UnitfulParams(β = 24.0u"hr", α = 3.0u"km/hr")


│ Row │ Symbol │ Value    │ Unit     │
│     │ Symbol │ Float64  │ Unitful… │
├─────┼────────┼──────────┼──────────┤
│ 1   │ α      │ 0.833333 │ m s⁻¹    │
│ 2   │ β      │ 1.0      │ d        │

Unpacking parameters that have units first converts them to SI units.

In [11]:
@unpack β = p
β

86400.0

and `β` (which is equal to 1 day) here is expressed in seconds after being unpacked.

## Initial values

Another useful feature is to set initial (or default) values.
Again, you **must** import the functions for them to work properly

In [12]:
import AIBECS: @initial_value, initial_value
@initial_value struct ParamsWithInitialValue{T} <: AbstractParameters{T}
    α::T | 1.0
    β::T | 2.0
end

initial_value (generic function with 11 methods)

This is handy in many applications.

You can instantiate `p` with the initial values as, well, its values.

In [13]:
p = ParamsWithInitialValue()


│ Row │ Symbol │ Value   │ Initial value │
│     │ Symbol │ Float64 │ Float64       │
├─────┼────────┼─────────┼───────────────┤
│ 1   │ α      │ 1.0     │ 1.0           │
│ 2   │ β      │ 2.0     │ 2.0           │

You could also just set one parameter to a different value

In [14]:
p = ParamsWithInitialValue(β = 10.0)


│ Row │ Symbol │ Value   │ Initial value │
│     │ Symbol │ Float64 │ Float64       │
├─────┼────────┼─────────┼───────────────┤
│ 1   │ α      │ 1.0     │ 1.0           │
│ 2   │ β      │ 10.0    │ 2.0           │

## Combining initial values and units

You can combine both features in parameters.

In [15]:
@units @initial_value struct UnitfulParamsWithInitialValue{T} <: AbstractParameters{T}
    α::T | 1.0 | u"m/s"
    β::T | 2.0 | u"d"
end

units (generic function with 13 methods)

And instantiate `p` from just one parameter with its unit

In [16]:
p = UnitfulParamsWithInitialValue(β = 1.0u"yr")

┌ Warning: Symbol yr was found in multiple registered unit modules.
│ We will use the one from UnitfulAstro.
└ @ Unitful /Users/runner/.julia/packages/Unitful/PZjMS/src/user.jl:589



│ Row │ Symbol │ Value   │ Initial value │ Unit     │
│     │ Symbol │ Float64 │ Float64       │ Unitful… │
├─────┼────────┼─────────┼───────────────┼──────────┤
│ 1   │ α      │ 1.0     │ 1.0           │ m s⁻¹    │
│ 2   │ β      │ 365.25  │ 2.0           │ d        │

## Optimizable parameters

In [17]:
import AIBECS: @flattenable, flattenable
@flattenable struct OptimizableParams{T} <: AbstractParameters{T}
    α::T | true
    β::T | false
    γ::T | true
end

flattenable (generic function with 12 methods)

Then the "flattenable" parameters will be the only ones to remain when converting `OptimizableParams` to a vector

In [18]:
p = OptimizableParams(1.0, 2.0, 3.0)
v = vec(p)

2-element Array{Float64,1}:
 1.0
 3.0

The `vec` function uses the `@unpack` function in AIBECS, so that units are converted when vectorizing.
Here is an example of that by first combining `units` and `flattenable`.

In [19]:
@flattenable @units @initial_value struct OptimizableParamsWithUnits{T} <: AbstractParameters{T}
    α::T | 1.0 | u"m/s" | true
    β::T | 2.0 | u"d"   | false
    γ::T | 3.0 | u"km"  | true
end
p = OptimizableParamsWithUnits()


│ Row │ Symbol │ Value   │ Initial value │ Unit     │ Optimizable │
│     │ Symbol │ Float64 │ Float64       │ Unitful… │ Bool        │
├─────┼────────┼─────────┼───────────────┼──────────┼─────────────┤
│ 1   │ α      │ 1.0     │ 1.0           │ m s⁻¹    │ 1           │
│ 2   │ β      │ 2.0     │ 2.0           │ d        │ 0           │
│ 3   │ γ      │ 3.0     │ 3.0           │ km       │ 1           │

And then vectorizing the parameters

In [20]:
vec(p)

2-element Array{Float64,1}:
    1.0
 3000.0

Note how `γ` (the third parameter, but the second flattenable one), is converted to meters.

## Other features

Coming soon!

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*